In [1]:
import numpy as np
import time
from lidsor_filter import filtering_lidsor_cpp
from glob import glob
import open3d as o3d
import pypatchworkpp

In [2]:
bin_path_list = glob(f"/Users/yukiya/ws/mini/*.bin")
bin_path = bin_path_list[0]


points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
print(points[:, 3])

[0.         0.         0.         ... 0.13636364 0.2090909  0.27272728]


In [14]:
# 地面点群の推定
params = pypatchworkpp.Parameters()
PatchworkPLUSPLUS = pypatchworkpp.patchworkpp(params)
PatchworkPLUSPLUS.estimateGround(points)
ground_idx      = PatchworkPLUSPLUS.getGroundIndices()
nonground_idx   = PatchworkPLUSPLUS.getNongroundIndices()
ground_points = points[ground_idx]
nonground_points = points[nonground_idx]


# 雑音除去
filtered_points = np.array(filtering_lidsor_cpp(nonground_points, scaling_factor=110.0), dtype=np.float64)

# ground_pointsを統合
merged_points = np.concatenate([ground_points, filtered_points], axis=0)

PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE


In [15]:
merged_pcd = o3d.geometry.PointCloud()
merged_pcd.points = o3d.utility.Vector3dVector(merged_points[:, :3])
# merged_pcd.colors = o3d.utility.Vector3dVector(merged_points[:, 3])
o3d.visualization.draw_geometries([merged_pcd])


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [8]:

ground_idx = PatchworkPLUSPLUS.getGroundIndices()
ground_pcd = o3d.geometry.PointCloud()

ground_intensity = points[ground_idx][:, 3]

# Create colors array, default to white
colors = np.ones((ground_intensity.shape[0], 3))

# Set points with intensity below threshold to black
threshold = 0.01 / 110.0  # Set your desired threshold here
colors[ground_intensity <= threshold] = [0, 0, 0]  # Black color for low intensity

ground_pcd.points = o3d.utility.Vector3dVector(points[ground_idx][:, :3])
ground_pcd.colors = o3d.utility.Vector3dVector(colors)  # Assign colors to the point cloud
# o3d.visualization.draw_geometries([ground_pcd])


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
